# KCA : KCA Ingredient List

## Import Module

In [1]:
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
from tqdm import tqdm_notebook
from bs4 import BeautifulSoup
from html_table_parser import parser_functions
import requests

## Crawling

In [ ]:
# kcia의 성분사전 테이블 형태로 빈 데이터 프레임을 미리 하나 만든다
cols = ['성분코드', '성분명', '영문명', 'CAS No', '구명칭']
kca_ing_name_df = pd.DataFrame(columns=cols)
kca_ing_name_df

In [ ]:
url = "https://kcia.or.kr/cid/search/ingd_list.php"
response = requests.get(url)

html = response.text
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
total = int(soup.find('b','green').text.replace(',','')) # 전체 자료수
last_page = 0 

# 한 페이지당 10개씩 기재되었음
if total % 10 != 0: # 10의 정수가 아니면 한 페이지 추가
    last_page = (total // 10) + 1
else:
    last_page = total // 10

In [ ]:
for num in range(1, last_page+1): # page_num을 활용
    url = f"https://kcia.or.kr/cid/search/ingd_list.php?page={num}" # url이 뒤의 page num만 바뀜
    response = requests.get(url)

    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

    data = soup.find('table',{"class" : "bbs_list"}) # class가 bbs_list인 테이블을 찾아와서
    data = parser_functions.make2d(data)[1:] # parser_functions의 make2d로 받아온다

    tmp_df = pd.DataFrame(data, columns=cols) # 임시 데이터 프레임을 만들어 각 페이지 별 테이블 정보를 담아서
    kca_ing_name_df = pd.concat([kca_ing_name_df ,tmp_df]) # 위에 만들어 놓은 빈 데이터프레임이랑 concat 하는 식으로 누적시킨다

# kca_ing_name_df

In [ ]:
kca_ing_name_df = kca_ing_name_df.astype({'성분코드':'int'})
kca_ing_name_df = kca_ing_name_df[['성분코드', '성분명', '영문명']].set_index('성분코드').sort_index()
kca_ing_name_df.reset_index(inplace=True)

In [ ]:
# inci-decoer에 성분을 검색할 때, format이 맞지 않는 표기명들이 있었다.
# 괄호와 그 안에 들어있는 내용을 제거하자
import re
pattern = r'\([^)]*\)'

for idx, row in kca_ing_name_df.iterrows():
    tmp = kca_ing_name_df.iloc[idx]['영문명']
    try:
        if '(' in tmp:
            txt = re.sub(pattern=pattern, repl='', string= tmp)
            txt = ' '.join(txt.split())
            kca_ing_name_df.iloc[idx,2] = txt
    except:
        pass

In [ ]:
kca_ing_name_df['formatted_영문명'] = kca_ing_name_df['영문명'].str.lower().str.replace(" ","-") # inci-decoder에 검색가능한 format으로 변경하여 컬럼 추가

In [ ]:
kca_ing_name_df.to_csv('kca_ing_name.csv')